### Import libraries

In [1]:
from model import model, losses, dot_prod_attention
from data import data_generation, batch_creator, gp_kernels
from keras.callbacks import ModelCheckpoint
from helpers import helpers, masks
from inference import infer
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
import tensorflow as tf
import numpy as np
import matplotlib 
import time
import keras

Using TensorFlow backend.


In [2]:
save_dir = '/Users/omernivron/Downloads/GPT'

In [3]:
pad_pos_tr, pad_pos_te, pad_y_fren_tr, pad_y_fren_te, _, df_te = data_generation.data_generator_for_gp_mimick_gpt(50000, gp_kernels.rbf_kernel)

In [4]:
pp = masks.position_mask(pad_pos_tr)
pp_te = masks.position_mask(pad_pos_te)

In [23]:
loss_object = tf.keras.losses.MeanSquaredError()
train_loss = tf.keras.metrics.Mean(name='train_loss')
test_loss = tf.keras.metrics.Mean(name='test_loss')
r_sq_tr = tfa.metrics.RSquare(dtype = tf.float64)
r_sq_te = tfa.metrics.RSquare()

In [24]:
@tf.function
def train_step(pos, tar, pos_mask):
    '''
    A typical train step function for TF2. Elements which we wish to track their gradient
    has to be inside the GradientTape() clause. see (1) https://www.tensorflow.org/guide/migrate 
    (2) https://www.tensorflow.org/tutorials/quickstart/advanced
    ------------------
    Parameters:
    pos (np array): array of positions (x values) - the 1st/2nd output from data_generator_for_gp_mimick_gpt
    tar (np array): array of targets. Notice that if dealing with sequnces, we typically want to have the targets go from 0 to n-1. The 3rd/4th output from data_generator_for_gp_mimick_gpt  
    pos_mask (np array): see description in position_mask function
    ------------------    
    '''
    tar_inp = tar[:, :-1]
    tar_real = tar[:, 1:]
    combined_mask_tar = masks.create_masks(tar_inp)
    with tf.GradientTape(persistent=True) as tape:
        pred, pred_sig = decoder(pos, tar_inp, True, pos_mask, combined_mask_tar)
#         print('pred: ')
#         tf.print(pred)

        loss = losses.loss_function(tar_real, pred, pred_sig)


    gradients = tape.gradient(loss, decoder.trainable_variables)
#     tf.print(gradients)
# Ask the optimizer to apply the processed gradients.
    optimizer_c.apply_gradients(zip(gradients, decoder.trainable_variables))
    train_loss(loss)
#     b = decoder.trainable_weights[0]
#     tf.print(tf.reduce_mean(b))

In [25]:
@tf.function
def test_step(pos_te, tar_te, pos_mask_te):
    '''
    
    ---------------
    Parameters:
    pos (np array): array of positions (x values) - the 1st/2nd output from data_generator_for_gp_mimick_gpt
    tar (np array): array of targets. Notice that if dealing with sequnces, we typically want to have the targets go from 0 to n-1. The 3rd/4th output from data_generator_for_gp_mimick_gpt  
    pos_mask_te (np array): see description in position_mask function
    ---------------
    
    '''
    tar_inp_te = tar_te[:, :-1]
    tar_real_te = tar_te[:, 1:]
    combined_mask_tar_te = masks.create_masks(tar_inp_te)
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
    pred, pred_sig = decoder(pos_te, tar_inp_te, False, pos_mask_te, combined_mask_tar_te)
    t_loss = losses.loss_function(tar_real_te, pred, pred_sig)
    test_loss(t_loss)
#     r_sq_te.update_state(tar_real_te, pred)

In [26]:
tf.keras.backend.set_floatx('float64')

In [ ]:
if __name__ == '__main__':
    writer = tf.summary.create_file_writer(save_dir + '/logs/')
    optimizer_c = tf.keras.optimizers.Adam()
    decoder = model.Decoder(16)
    EPOCHS = 50
    batch_s  = 128
    run = 0; step = 0
    num_batches = int(pad_y_fren_tr.shape[0] / batch_s)
    tf.random.set_seed(1)    
    checkpoint = tf.train.Checkpoint(optimizer = optimizer_c, model = decoder)
    main_folder = "/Users/omernivron/Downloads/GPT/ckpt/check_"
    folder = main_folder + str(run); helpers.mkdir(folder)

    with writer.as_default():
        for epoch in range(EPOCHS):
            start = time.time()

            for batch_n in range(num_batches):
                batch_pos_tr, batch_tar_tr, batch_pos_mask, _ = batch_creator.create_batch_gp_mim_2(pad_pos_tr, pad_y_fren_tr, pp)
                # batch_tar_tr shape := 128 X 59 = (batch_size, max_seq_len)
                # batch_pos_tr shape := 128 X 59 = (batch_size, max_seq_len)
                train_step(batch_pos_tr, batch_tar_tr, batch_pos_mask)

                if batch_n % 50 == 0:
                    batch_pos_te, batch_tar_te, batch_pos_mask_te, _ = batch_creator.create_batch_gp_mim_2(pad_pos_te, pad_y_fren_te, pp_te)
                    test_step(batch_pos_te, batch_tar_te, batch_pos_mask_te)
                    helpers.print_progress(epoch, batch_n, train_loss.result(), test_loss.result(), r_sq_tr.result())
                    helpers.tf_summaries(run, step, train_loss.result(), test_loss.result(), r_sq_tr.result(), r_sq_te.result())
                    checkpoint.save(folder + '/')
                step += 1

            print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

New folder /Users/omernivron/Downloads/GPT/ckpt/check_0
Epoch 0 batch 0 train Loss 3.5894 test Loss 1.6112 with R^2 nan
Epoch 0 batch 50 train Loss 1.2462 test Loss 0.9822 with R^2 nan
Epoch 0 batch 100 train Loss 0.9187 test Loss 0.7395 with R^2 nan
Epoch 0 batch 150 train Loss 0.7620 test Loss 0.6066 with R^2 nan
Epoch 0 batch 200 train Loss 0.6627 test Loss 0.5288 with R^2 nan
Epoch 0 batch 250 train Loss 0.5921 test Loss 0.4552 with R^2 nan
Epoch 0 batch 300 train Loss 0.5352 test Loss 0.3995 with R^2 nan
Time taken for 1 epoch: 458.16467475891113 secs

Epoch 1 batch 0 train Loss 0.5237 test Loss 0.3616 with R^2 nan
Epoch 1 batch 50 train Loss 0.4806 test Loss 0.3317 with R^2 nan
Epoch 1 batch 100 train Loss 0.4458 test Loss 0.3057 with R^2 nan
Epoch 1 batch 150 train Loss 0.4171 test Loss 0.2847 with R^2 nan
Epoch 1 batch 200 train Loss 0.3923 test Loss 0.2665 with R^2 nan
Epoch 1 batch 250 train Loss 0.3706 test Loss 0.2526 with R^2 nan
Epoch 1 batch 300 train Loss 0.3511 test Lo

tar = pad_y_fren_te[:, 1:]



In [34]:
1 - (0.0165 / sum((tar[:, 5] - np.mean(tar[:, 5]))**2) / len(tar[:, 5]))

0.999999999833459

In [23]:
tar - np.mean(tar, 0)

array([[-0.23126513, -0.22783515, -0.15818167, ..., -0.12317099,
        -0.32767493, -0.21421905],
       [-0.70544454, -0.73005885, -0.6830323 , ..., -0.74098858,
        -0.59824057, -0.81173125],
       [ 0.10899795,  0.18115021,  0.22626579, ...,  0.20117891,
         0.25371165,  0.13876939],
       ...,
       [-0.38531545, -0.30379112, -0.27217713, ..., -0.26784763,
        -0.299351  , -0.39355877],
       [ 1.2178523 ,  1.30296639,  1.24647646, ...,  1.19968792,
         1.19953355,  1.18947239],
       [-1.21407981, -1.19715032, -1.27335662, ..., -1.23137991,
        -1.22176898, -1.21790715]])

In [12]:
tar.shape

(10000, 58)

In [16]:
np.mean(tar[:, 0])

0.005712169343526802

In [21]:
sum((tar[:, 0] - np.mean(tar[:, 0]))**2 )/ 10000

1.0170927027632388

In [21]:
sum(sum((tar - np.mean(tar))**2)) / (tar.shape[0] * tar.shape[1])

0.9916270624993678

In [ ]:
pos = df_te[560, :].reshape(1, -1)

In [ ]:
tar = df_te[561, :39].reshape(1, -1)

In [ ]:
df_te[561, :]

In [ ]:
a = inference(pos, tar, 20)

In [ ]:
with matplotlib.rc_context({'figure.figsize': [10,2.5]}):
    plt.scatter(pos[:, :39], tar[:, :39], c='black')
    plt.scatter(pos[:, 39:58], a[39:])
    plt.scatter(pos[:, 39:58], df_te[561, 39:58], c='red')

In [ ]:
# tf.data.Dataset(tf.Tensor(pad_pos_tr, value_index = 0 , dtype = tf.float32))